In [1]:
import pandas as pd
import numpy as np
import pickle
import random
from matplotlib import pyplot as plt

# !pip uninstall mlbugdetection
# import mlbugdetection
from mlbugdetection.monotonic import monotonicity_mse, check_monotonicity
from mlbugdetection.critical_values import find_critical_values, highest_and_lowest_indexes
from mlbugdetection.calibration import calibration_check
from mlbugdetection.sanity import sanity_check

In [2]:
#manter plots salver disco
#classe 
# help(sanity_check)

In [3]:
with open('models/XGBoost/XGBoost.pkl', 'rb') as f:
    XGBoost = pickle.load(f)
with open('models/LogisticRegression/LogisticRegression.pkl', 'rb') as f:
    LR = pickle.load(f)
with open('models/RandomForest/RandomForest.pkl', 'rb') as f:
    RF = pickle.load(f)
with open('models/SVM/SVMTitanic.pkl', 'rb') as f:
    SVM = pickle.load(f)
models = [XGBoost, LR, RF]


C:\Users\willi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\willi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\willi\AppData\Local\Packa

In [41]:
df = pd.read_csv('../datasets/fraud_new.csv')
df_titanic = pd.read_csv('../datasets/TitanicClean.csv')
example = df.sample(1)
examples = df.sample(100)
# example = example.drop('isFraud', axis = 1)

In [13]:
example_titanic = df_titanic.sample(1)

Teste Sanity Check

In [66]:
import pickle
import pandas as pd
from .analysis_report import AnalysisReport

def check_type_input_model(model):
    ''' Check the type of the input model and returns the model object '''
    if type(model) == str:
        with open(model, 'rb') as f:
            model = pickle.load(f)
    return model

def sanity_check(model, samples, target):
    '''Sanity Test
        Analyzes the sanity of a model with samples and 
        return a bool that represents if the tests passed or not.

    Parameters
    ----------
    model : sklearn model or str
        The model to be used for prediction. Could be a model object or a path to a model file.

    samples : pandas DataFrame
        The samples to be used for prediction, which the model
        need to predict correctly. 

    target : str
        The name of the column containing the target variable.

    Returns
    -------
    bool True if the model is sane, False otherwise.
    '''
    model = check_type_input_model(model)

    result = model.predict(samples.drop(target, axis=1))
    original = samples[target]

    result = pd.Series(result).reset_index(drop=True)
    origin = original.reset_index(drop=True)
    values = result == origin

    if len(values.value_counts().index) == 2:
        return False

    return values[0]

def sanity_check_with_indexes(model, samples, target):
    '''Sanity Test With Indexes
        Analyzes the sanity of a model with samples and 
        shows a Analysis Report that shows if the tests passed or not.
        If the tests failed, it will show the indexes of the samples that were misclassified.

    Parameters
    ----------
    model : sklearn model or str
        The model to be used for prediction. Could be a model object or a path to a model file.

    samples : pandas DataFrame
        The samples to be used for prediction, which the model
        need to predict correctly. 

    target : str
        The name of the column containing the target variable.

    Returns
    -------
     AnalysisReport object with following attributes:
        For more information:
        >>> from mlbugdetection.analysis_report import AnalysisReport
        >>> help(AnalysisReport)

    model_name : str
        Name of the model being analysed.
    
    analysed_feature : str
        Name of the feature being analysed.

    metrics : dictionary
        Dictionary with all the calculated metrics, such as:
        
        'sanity' : bool
            If the model is sane or not.

        'sanity_indexes': List
            List of indexes of the samples that were misclassified.

    '''
    model = check_type_input_model(model)
    report = AnalysisReport()

    result = model.predict(samples.drop(target, axis=1))
    original = samples[target]

    result = pd.Series(result).reset_index(drop=True)
    origin = original.reset_index(drop=True)
    values = result == origin
    
    report.model_name = type(model).__name__
    report.analysed_feature = target

    if len(values.value_counts().index) == 2:
        report.metrics["sanity"] = False
        report.metrics["sanity_indexes"] = values[values==False].index.to_list()
        return report

    report.metrics["sanity"] = True
    report.metrics["sanity_indexes"] = []
    return report

sanity_check_with_indexes(XGBoost, df, 'isFraud')

In [74]:
idk = df.iloc[221:250,:]
sanity_check(XGBoost, idk, 'isFraud')
sanity_check_with_indexes(XGBoost, idk, 'isFraud')

Others analysis

In [ ]:
for c in df_titanic.columns:
    if c != 'isFraud':
        minV = df_titanic[c].min()
        maxV = df_titanic[c].max()
        teste = check_monotonicity(c,minV,maxV,example_titanic, "models/SVM/SVMTitanic.pkl", 150)
        # teste = check_monotonicity(c,minV,maxV,example_titanic, SVM, 150)
    break

In [ ]:
print(teste)
teste.save_graphs()

In [ ]:
print(teste.errors)
print(teste.warnings)
print(teste.metrics)
print(teste.model_info)

In [ ]:
for c in df_titanic.columns:
    if c != 'isFraud':
        minV = 100
        maxV = 100
        find_critical_values(SVM, example_titanic, c, minV, maxV, step = 1000 )

In [ ]:
for c in df.columns:
    if c != 'isFraud':
        minV = df[c].min()
        maxV = df[c].max()
        for model in models:
            teste3 = find_critical_values(model, example, c, minV, maxV )
    break

In [ ]:
for c in df.columns:
    if c != 'isFraud':
        minV = df[c].min()
        maxV = df[c].max()
        for model in models:
            # check_monotonicity(c,minV,maxV,example, model, 150, plot_graph=True)
            check_monotonicity(c,minV,maxV,example, model, 150)

In [ ]:
for model in models:
    teste2 = calibration_check("isFraud", model, df)
    break

In [ ]:
teste2.save_graphs()

In [ ]:
teste2.graphs